# Customizing Memory Management 
This tutorial goes over how to implement a custom memory class in Letta, which allows you to customize how memory is organized (via `Block` objects) and also how memory is maintained (through memory editing tools). 


## Section 0: Setup a MemGPT client 

In [2]:
from letta import create_client 

client = create_client() 

In [3]:
from letta import LLMConfig, EmbeddingConfig

client.set_default_llm_config(LLMConfig.default_config("gpt-4o-mini")) 
client.set_default_embedding_config(EmbeddingConfig.default_config("text-embedding-ada-002")) 

## Section 1: Memory Blocks 
Core memory consists of multiple memory *blocks*. A block represents a section of the LLM's context window, reservered to store the block's value (with an associated character limit). Blocks are persisted in the DB, so can be re-used or also shared accross agents. 

## Understanding `ChatMemory`

In [4]:
from letta import ChatMemory 

In [5]:
chat_memory = ChatMemory(
    human="Name: Bob", 
    persona="You are a helpful assistant"
)

#### Memory blocks 
A memory class consists of a list of `Block` objects (labeled with a block name), as well as function definitions to edit these blocks. These blocks each represent a section of the context window reserved for memory. 

In [8]:
chat_memory.get_blocks()

[Block(value='You are a helpful assistant', limit=2000, template_name=None, template=False, label='persona', description=None, metadata_={}, user_id=None, id='block-865bef7d-ab60-4e73-a376-2f34357cfaa0'),
 Block(value='Name: Bob', limit=2000, template_name=None, template=False, label='human', description=None, metadata_={}, user_id=None, id='block-45401bef-cd7c-492e-ae7e-50ab501c0c6f')]

In [9]:
chat_memory.get_block("human")

Block(value='Name: Bob', limit=2000, template_name=None, template=False, label='human', description=None, metadata_={}, user_id=None, id='block-45401bef-cd7c-492e-ae7e-50ab501c0c6f')

#### Memory editing functions  
The `Memory` class also consists of functions for editing memory, which are provided as tools to the agent (so it can call them to edit memory). The `ChatMemory` class provides `core_memory_append` and `core_memory_append` functions. 

In [10]:
import inspect

In [11]:
print(inspect.getsource(chat_memory.core_memory_append))

    def core_memory_append(self: "Agent", label: str, content: str) -> Optional[str]:  # type: ignore
        """
        Append to the contents of core memory.

        Args:
            label (str): Section of the memory to be edited (persona or human).
            content (str): Content to write to the memory. All unicode (including emojis) are supported.

        Returns:
            Optional[str]: None is always returned as this function does not produce a response.
        """
        current_value = str(self.memory.get_block(label).value)
        new_value = current_value + "\n" + str(content)
        self.memory.update_block_value(label=label, value=new_value)
        return None



#### Context compilation 
Each time the LLM is called (for each reasoning step of the agent), the memory is "compiled" into a context window representation. 

In [12]:
chat_memory.get_prompt_template()

'{% for block in memory.values() %}<{{ block.label }} characters="{{ block.value|length }}/{{ block.limit }}">\n{{ block.value }}\n</{{ block.label }}>{% if not loop.last %}\n{% endif %}{% endfor %}'

In [13]:
chat_memory.compile()

'<persona characters="27/2000">\nYou are a helpful assistant\n</persona>\n<human characters="9/2000">\nName: Bob\n</human>'

## Section 2: Defining a custom memory module 
In the previous example, we used a built in `ChatMemory` class which has a `human` and `persona` field in the memory to allow the agent to save important information in a 1:1 chat, and also used the `BasicBlockMemory` to customize the memory blocks. 

In the section, we'll go over how to define a custom memory class, including how to implement memory editing tools. We'll do this by implementing a `TaskMemory` class, which has a section of memory that is reserved for a list of tasks that can be pushed and popped form. 

### Defining a memory module


In [21]:
from letta import ChatMemory, Block 
from typing import Optional, List
import json

class TaskMemory(ChatMemory): 

    def __init__(self, human: str, persona: str, tasks: List[str]): 
        super().__init__(human=human, persona=persona, limit=2000) 
        self.link_block( 
            Block(
                limit=2000, 
                value=json.dumps(tasks), 
                label="tasks"
            )
        )

    def task_queue_push(self: "Agent", task_description: str):
        """
        Push to a task queue stored in core memory. 

        Args:
            task_description (str): A description of the next task you must accomplish. 
            
        Returns:
            Optional[str]: None is always returned as this function 
            does not produce a response.
        """
        import json
        tasks = json.loads(self.memory.get_block("tasks").value)
        tasks.append(task_description)
        self.memory.update_block_value("tasks", json.dumps(tasks))
        return None

    def task_queue_pop(self: "Agent"):
        """
        Get the next task from the task queue 
 
        Returns:
            Optional[str]: The description of the task popped from the 
            queue, if there are still tasks in queue. Otherwise, returns
            None (the task queue is empty)
        """
        import json
        tasks = json.loads(self.memory.get_block("tasks").value)
        if len(tasks) == 0: 
            return None
        task = tasks[0]
        print("CURRENT TASKS: ", tasks)
        self.memory.update_block_value("tasks", json.dumps(tasks[1:]))
        return task


### Creating an agent with custom `TaskMemory`

In [22]:
task_agent_name = "task_agent"

task_agent_state = client.create_agent(
    name=task_agent_name, 
    system = open("data/task_queue_system_prompt.txt", "r").read(),
    memory=TaskMemory(
        human="My name is Sarah", 
        persona="You are an agent that must clear its tasks.", 
        tasks=[]
    )
)

In [23]:
response = client.send_message(
    agent_id=task_agent_state.id, 
    role="user", 
    message="Add 'start calling me Charles' and 'tell me a haiku about my name' as two seperate tasks."
)
response

/Users/sarahwooders/repos/letta/letta/helpers/tool_rule_solver.py:70: UserWarning: User provided tool rules and execution state resolved to no more possible tool calls.
  warnings.warn(message)


CURRENT TASKS:  ['start calling me Charles', 'tell me a haiku about my name']


LettaResponse(messages=[InternalMonologue(id='message-985cc68a-3276-4444-a1ad-e591ac224cf0', date=datetime.datetime(2024, 11, 7, 4, 37, 6, 166293, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="User requested two new tasks. I'll add them one by one."), FunctionCallMessage(id='message-985cc68a-3276-4444-a1ad-e591ac224cf0', date=datetime.datetime(2024, 11, 7, 4, 37, 6, 166293, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='task_queue_push', arguments='{\n  "task_description": "start calling me Charles",\n  "request_heartbeat": true\n}', function_call_id='call_ejclsdkkYxd1mCeqOv2nd5PP')), FunctionReturn(id='message-a049b96e-aba0-4f84-85bb-3e644704036e', date=datetime.datetime(2024, 11, 7, 4, 37, 6, 167388, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:37:06 PM PST-0800"\n}', status='success', function_call_id='call_ejclsdkkYxd1mCeqOv2nd5PP'), InternalMonologue(id='message-1ef7118b-0e40-4827-8bb8-f2d828f6e14d', date=datetime.datetime(2024, 11, 7, 4, 37, 8, 830449, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="Now I'll add the second task about a haiku."), FunctionCallMessage(id='message-1ef7118b-0e40-4827-8bb8-f2d828f6e14d', date=datetime.datetime(2024, 11, 7, 4, 37, 8, 830449, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='task_queue_push', arguments='{\n  "task_description": "tell me a haiku about my name",\n  "request_heartbeat": true\n}', function_call_id='call_fAUwIS8LMdIXSYl13dZMHAH5')), FunctionReturn(id='message-5dd1ecc9-2c04-40e4-8d90-a0009d43e5fe', date=datetime.datetime(2024, 11, 7, 4, 37, 8, 832851, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:37:08 PM PST-0800"\n}', status='success', function_call_id='call_fAUwIS8LMdIXSYl13dZMHAH5'), InternalMonologue(id='message-0687755d-180f-4399-83c3-0ac2493f7341', date=datetime.datetime(2024, 11, 7, 4, 37, 9, 840806, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="Tasks added. Now I'll clear the task queue next."), FunctionCallMessage(id='message-0687755d-180f-4399-83c3-0ac2493f7341', date=datetime.datetime(2024, 11, 7, 4, 37, 9, 840806, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='task_queue_pop', arguments='{\n  "request_heartbeat": true\n}', function_call_id='call_x44aL8FIGcMcJlkuO5MeYoqo')), FunctionReturn(id='message-b68af297-3d9d-451c-a073-313474a5c911', date=datetime.datetime(2024, 11, 7, 4, 37, 9, 847964, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "start calling me Charles",\n  "time": "2024-11-06 08:37:09 PM PST-0800"\n}', status='success', function_call_id='call_x44aL8FIGcMcJlkuO5MeYoqo'), InternalMonologue(id='message-e7685454-3424-4d79-8294-07b2c21e911d', date=datetime.datetime(2024, 11, 7, 4, 37, 11, 76376, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='I need to adjust how I address the user next.'), FunctionCallMessage(id='message-e7685454-3424-4d79-8294-07b2c21e911d', date=datetime.datetime(2024, 11, 7, 4, 37, 11, 76376, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Got it, Charles! From now on, I\'ll call you by that name."\n}', function_call_id='call_592kDLiCB5Rt0nY4nHFteE3r')), FunctionReturn(id='message-dab1e366-1d89-4c71-b94e-d4ae66e37402', date=datetime.datetime(2024, 11, 7, 4, 37, 11, 77104, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:37:11 PM PST-0800"\n}', status='success', function_call_id='call_

In [24]:
response = client.send_message(
    agent_id=task_agent_state.id, 
    role="user", 
    message="complete your tasks"
)
response

CURRENT TASKS:  ['tell me a haiku about my name']


LettaResponse(messages=[InternalMonologue(id='message-27ca6f0a-0751-4090-aac6-68ae38f5ad35', date=datetime.datetime(2024, 11, 7, 4, 37, 13, 96373, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Checking next task to complete from the queue.'), FunctionCallMessage(id='message-27ca6f0a-0751-4090-aac6-68ae38f5ad35', date=datetime.datetime(2024, 11, 7, 4, 37, 13, 96373, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='task_queue_pop', arguments='{\n  "request_heartbeat": true\n}', function_call_id='call_UzwHlQkPuyQUyBecvU5cVvab')), FunctionReturn(id='message-f8cecbb0-bdf1-46c3-8d2e-9cfe35fd392e', date=datetime.datetime(2024, 11, 7, 4, 37, 13, 102275, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "tell me a haiku about my name",\n  "time": "2024-11-06 08:37:13 PM PST-0800"\n}', status='success', function_call_id='call_UzwHlQkPuyQUyBecvU5cVvab'), InternalMonologue(id='message-e4e3dfcc-56c6-437e-8e1f-4e14eb1e7548', date=datetime.datetime(2024, 11, 7, 4, 37, 14, 444273, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue='Completing the haiku task for Charles. Here goes!'), FunctionCallMessage(id='message-e4e3dfcc-56c6-437e-8e1f-4e14eb1e7548', date=datetime.datetime(2024, 11, 7, 4, 37, 14, 444273, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "Here’s a haiku for you, Charles:\\n\\nWith strength, you embrace,\\nWhispers of your name surround,\\nCharles, calm like the sea."\n}', function_call_id='call_ykUJiQpCi0zLe4XPuQyWPvJ3')), FunctionReturn(id='message-65f2a5de-5577-47e3-994a-e3dc17bd7fdc', date=datetime.datetime(2024, 11, 7, 4, 37, 14, 447766, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:37:14 PM PST-0800"\n}', status='success', function_call_id='call_ykUJiQpCi0zLe4XPuQyWPvJ3')], usage=LettaUsageStatistics(completion_tokens=96, prompt_tokens=6409, total_tokens=6505, step_count=2))

In [25]:
response = client.send_message(
    agent_id=task_agent_state.id, 
    role="user", 
    message="keep going"\
)
response

LettaResponse(messages=[InternalMonologue(id='message-b9ff3fe6-4850-40f6-9c75-c37b0bad100f', date=datetime.datetime(2024, 11, 7, 4, 37, 16, 138739, tzinfo=datetime.timezone.utc), message_type='internal_monologue', internal_monologue="Let's keep the conversation flowing!"), FunctionCallMessage(id='message-b9ff3fe6-4850-40f6-9c75-c37b0bad100f', date=datetime.datetime(2024, 11, 7, 4, 37, 16, 138739, tzinfo=datetime.timezone.utc), message_type='function_call', function_call=FunctionCall(name='send_message', arguments='{\n  "message": "What would you like to talk about next, Charles? I’m all ears!"\n}', function_call_id='call_ZTuH5CZlz6At9Y1ltVBttNNj')), FunctionReturn(id='message-c3b31082-ce78-42ba-9434-7edec821c3dc', date=datetime.datetime(2024, 11, 7, 4, 37, 16, 146847, tzinfo=datetime.timezone.utc), message_type='function_return', function_return='{\n  "status": "OK",\n  "message": "None",\n  "time": "2024-11-06 08:37:16 PM PST-0800"\n}', status='success', function_call_id='call_ZTuH5CZlz6At9Y1ltVBttNNj')], usage=LettaUsageStatistics(completion_tokens=41, prompt_tokens=3431, total_tokens=3472, step_count=1))

In [26]:
client.get_in_context_memory(task_agent_state.id).get_block("tasks")

Block(value='[]', limit=2000, template_name=None, template=False, label='tasks', description=None, metadata_={}, user_id=None, id='block-288d04a9-e5c3-4da8-8746-89a728130b9a')